In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606694223806_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark=SparkSession.builder.appName('data_processing').getOrCreate()
df=spark.read.csv('s3://jmvalenciz-bigdata-bucket/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Columnas
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [5]:
# Número de filas
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6211

In [6]:
# Schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [7]:
# Descripción del dataframe
# Resultado en el archivo COVID_19_Statistics.txt
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-------------------+------------------------+--------+------------------+----+-----------+---------+-------------------+--------------------+--------------------+--------------------+
|summary|        ID de caso|   Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención|              Edad|Sexo|       Tipo|   Estado|País de procedencia|                 FIS|     Fecha de muerte|    Fecha recuperado|
+-------+------------------+------------------+-------------------+------------------------+--------+------------------+----+-----------+---------+-------------------+--------------------+--------------------+--------------------+
|  count|              6211|              6211|               6211|                    6211|    6211|              6211|6211|       6211|     6211|               6211|                6211|                6211|                5952|
|   mean|            3106.0|30742.691515053935|               null|         

In [10]:
# Eliminar columna Sexo
df.drop('Sexo').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|  Importado|  Leve|              Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-03-06 00:00:00|


In [14]:
# agregar columna NivelRiesgo
# datos tomados de https://espanol.cdc.gov/coronavirus/2019-ncov/need-extra-precautions/older-adults.html
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def nivelRiesgo(edad):
    if(edad <= 29):
        return 1
    elif(edad <= 39):
        return 4
    elif(edad <= 49):
        return 10
    elif(edad<= 64):
        return 30
    elif(edad<=74):
        return 90
    elif(edad<=84):
        return 220
    else:
        return 630
    
nivelRiesgo_udf=udf(nivelRiesgo,IntegerType())

df.withColumn('NivelRiesgo',nivelRiesgo_udf(df['Edad'])).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+-----------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |NivelRiesgo|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+-----------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia                   |2020-

In [54]:
# Algunos elementos de la columna "Sexo" están en minúscula por lo que esta
# parte corrige esto para que todo este en mayúscula
from pyspark.sql.types import StringType

def SexoToUppercase(str):
    if(str == "f"):
        return "F"
    elif(str == "m"):
        return "M"
    else:
        return str

SexoToUppercase_udf =udf(SexoToUppercase,StringType())

# CASOS
df.select("Sexo").distinct().show(10)

df.withColumn("Sexo",SexoToUppercase_udf(df['Sexo'])).select("Sexo").distinct().show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|Sexo|
+----+
|   M|
|   f|
|   F|
|   m|
+----+

+----+
|Sexo|
+----+
|   M|
|   F|
+----+

In [55]:
# Transformación de tipo
# El estado de un caso es de tipo string y se va a pasar a tipo entero

@udf
def EstadoToInt(estado):
    if(estado == "Leve" or estado == "leve"):
        return 1
    elif(estado == "Moderado"):
        return 2
    elif(estado == "Grave"):
        return 3
    elif(estado == "Fallecido"):
        return -1
    elif(estado == "N/A"):
        return 0
    else:
        return None

# Posibles valores
df.select("Estado").distinct().show()

df.withColumn("Estado", EstadoToInt(df.Estado)).select("Estado").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|   Estado|
+---------+
|     Leve|
|    Grave|
|Fallecido|
| Moderado|
|      N/A|
|     leve|
+---------+

+------+
|Estado|
+------+
|     3|
|     0|
|     1|
|    -1|
|     2|
+------+